In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve, classification_report, confusion_matrix
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback,CSVLogger
from keras.models import Sequential, load_model,Model , clone_model
from sklearn.cluster import KMeans,DBSCAN,AgglomerativeClustering
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dense, Reshape, Input
from keras.preprocessing.text import Tokenizer
from keras.losses import BinaryCrossentropy
from keras.backend import clear_session
from keras.initializers import Constant
from threading import current_thread
from keras.utils import plot_model
from sklearn.utils import shuffle
from keras.optimizers import Adam
from keras.utils import Sequence
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from numpy.linalg import norm
from functools import partial
from random import uniform
from numpy import asarray
from copy import deepcopy
import concurrent.futures
from keras import layers
import matplotlib as mpl
from keras import Model
from numpy import save
from numpy import load
import seaborn as sns
# import esprima as esp
from time import time
import pandas as pd
import numpy as np
import warnings
import tempfile
import logging
import urllib
import random
import pickle
import pydot
import torch
import math
import html
import nltk
import os
import re
import gc
warnings.filterwarnings("ignore")
np.random.seed(123)
plt.style.use('ggplot')
# mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
par_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On JavaScript File/Hiden"
model_info_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On JavaScript File/Results_TabuSearch3/Model info"
results_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On JavaScript File/Results_TabuSearch3"

In [ ]:
# par_path = "/content/drive/MyDrive/Colab Notebooks/Js_Contana/On JavaScript File/Hiden"
# model_info_path = "/content/drive/MyDrive/Colab Notebooks/Js_Contana/On JavaScript File/Results_TabuSearch4/Model info"
# results_path = "/content/drive/MyDrive/Colab Notebooks/Js_Contana/On JavaScript File/Results_TabuSearch4"

In [ ]:
class GarbageCollectorCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [ ]:
def _replaceitem(x):
    if type(x) is list:
        if x[0]<0.5:return 0.0
    else:
        x=float(x)
        if x<0.5:return 0.0
    return 1.0

In [ ]:
def save_object(obj, filename,path):
    """
    _ INPUT (obj) THE OBJECT WE NEED SAVW IT (filename) THE NAME OF OBJECT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [ ]:
def load_object(filename,path):
    """
    _ INPUT THE NAME OF OBJECT WE NEED LOAD IT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [ ]:
def create_model(shape=None,without_first_layer=False,i=0):
    """
    _INPUT (without_first_layer) IF WE WANT TO CREAT MODEL WITHOUT FIRST LSTM LAYER (shape) IF THE PREVIOS PAREMETER IS TRUE WE NEED DIFINE THE SHAPE FOR INPUT LAYER
    _OUTPUT THE MODEL
    """
    clear_session()
    model = Sequential()
    name_model = 'tabu_search_model_'+str(i)+'_.best.hdf5'
    model.add(layers.InputLayer(input_shape=shape))
    # TextCNN with 4 conv layers
    model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
    model.add(Conv1D(128, 15, activation='tanh'))
    model.add(Conv1D(128, 25, activation='tanh'))
    model.add(Conv1D(128, 35, activation='tanh'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    # model.load_weights(os.path.join(model_info_path,name_model))
    plot_model(model, to_file=os.path.join(model_info_path,name_model[:-10]+'.png'), show_shapes=True, show_layer_names=True)
    # plot_model(model, to_file=os.path.join(model_info_path,name_model[:-10]+'.svg'), show_shapes=True, show_layer_names=True)

    # model.summary()
    return model

In [ ]:
def plot_finall_results(Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train,F1_test,Ac_test,F1_train,Ac_train,method_name,step=1):
  title = '2TS-DFS method Vs '+method_name
  fig, (ax1, ax2) = plt.subplots(1, 2, layout='constrained', figsize=(15, 4))

  fig.suptitle(title)

  ax1.plot([i for i, x in enumerate(Best_F1_test) if i % step == 0],[x for i, x in enumerate(Best_F1_test) if i % step == 0], color="red",label="F1_Score_test")
  ax1.plot([i for i, x in enumerate(Best_Ac_test) if i % step == 0],[x for i, x in enumerate(Best_Ac_test) if i % step == 0], color="blue",label="Accurce_test")
  ax1.plot([i for i, x in enumerate(Best_F1_train) if i % step == 0],[x for i, x in enumerate(Best_F1_train) if i % step == 0], color="yellow",label="F1_Score_train")
  ax1.plot([i for i, x in enumerate(Best_Ac_train) if i % step == 0],[x for i, x in enumerate(Best_Ac_train) if i % step == 0], color="green",label="Accurce_train")

  ax1.legend(loc="best")
  ax1.set_xlabel('Chunk')
  ax1.set_ylabel('Results per Chunk')

  ax2.plot([i for i, x in enumerate(F1_test) if i % step == 0],[x for i, x in enumerate(F1_test) if i % step == 0], color="red",label="F1_Score_test")
  ax2.plot([i for i, x in enumerate(Ac_test) if i % step == 0],[x for i, x in enumerate(Ac_test) if i % step == 0], color="blue",label="Accurce_test")
  ax2.plot([i for i, x in enumerate(F1_train) if i % step == 0],[x for i, x in enumerate(F1_train) if i % step == 0], color="yellow",label="F1_Score_train")
  ax2.plot([i for i, x in enumerate(Ac_train) if i % step == 0],[x for i, x in enumerate(Ac_train) if i % step == 0], color="green",label="Accurce_train")

  ax2.legend(loc="best")
  ax2.set_xlabel('Chunk')
  ax2.set_ylabel('Results per Chunk')

  # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),title))
  # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),title+'.svg'),format='svg')
  plt.show()

In [ ]:
def selected_features_for_each_cluster(tabu,number_of_chunk):
  # set width of bar
  barWidth = 0.05
  fig = plt.subplots(figsize =(15, 5))
  # Set position of bar on X axis
  br1 = np.arange(len(tabu))
  # Make the plot
  plt.bar(br1, tabu,label ='2TS-DFS') # tabu
  # Adding Xticks
  plt.xlabel('cluster number', fontweight ='bold', fontsize = 10)
  plt.ylabel('The remaining dimensions after being reduced from 100', fontweight ='bold', fontsize = 8)
  plt.xticks([r + barWidth for r in range(len(tabu))],[str(i) for i,_ in enumerate(tabu)])
  # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"selected features for each cluster in chunk_"+str(number_of_chunk)))
  # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"selected features for each cluster in chunk_"+str(number_of_chunk)+".svg"),format='svg')
  plt.show()

In [ ]:
def _plot_results_for_chunk_number(chunk_number,f1=None,ac=None,pr=None,rc=None,step=None):

    plt.figure(figsize=(15, 5))

    if step is None:
      number = f1.index(max(f1)) if type(f1) is list else list(f1).index(max(f1))
      factors = find_factors(number) if number!=0 else [3,5]
      step = factors[1] if len(factors)==2 else factors[2]

    plt.title("Tabu Search results for chunk number "+str(chunk_number))
    XF = [x for i, x in enumerate(f1) if i % step == 0]
    XA = [x for i, x in enumerate(ac) if i % step == 0]
    XFT = [x for i, x in enumerate(pr) if i % step == 0]
    XAT = [x for i, x in enumerate(rc) if i % step == 0]
    save_object(XF, "F1_Score_test_"+str(chunk_number),results_path)
    save_object(XA, "Accurce_score_test_"+str(chunk_number),results_path)
    save_object(XFT, "F1_Score_train_"+str(chunk_number),results_path)
    save_object(XAT, "Accurce_score_train_"+str(chunk_number),results_path)

    plt.plot([i for i, x in enumerate(f1) if i % step == 0],XF, color="red",label="F1_Score_test")
    plt.plot([i for i, x in enumerate(ac) if i % step == 0],XA, color="blue",label="Accurce_score_test")
    plt.plot([i for i, x in enumerate(pr) if i % step == 0],XFT, color="yellow",label="F1_Score_train")
    plt.plot([i for i, x in enumerate(rc) if i % step == 0],XAT, color="green",label="Accurce_score_train")

    plt.scatter(f1.index(max(f1)) if type(f1) is list else list(f1).index(max(f1)), max(f1), color='orange', s=25)

    plt.vlines(x=f1.index(max(f1)) if type(f1) is list else list(f1).index(max(f1)), ymin=min([min(f1),min(f1),min(f1),min(f1)]),
               ymax=max([max(f1),max(ac),max(pr),max(rc)]), colors='black', lw=1)

    plt.hlines(y=max(f1), xmin=0,
            xmax=f1.index(max(f1)) if type(f1) is list else list(f1).index(max(f1)), colors='black', lw=1)

    plt.legend(loc="best")
    plt.xlabel('iteration')
    plt.ylabel('results per iteration')
    # plt.xscale('log',base=2)

    # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Tabu Search results for chunk number "+str(chunk_number)))
    # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Tabu Search results for chunk number "+str(chunk_number)+'.svg'),format='svg')
    plt.show()

In [ ]:
def plot_results_for_chunk_number(chunk_number,f1=None,ac=None,pr=None,rc=None,step=5):

    plt.figure(figsize=(15, 5))

    plt.title("Tabu Search results for chunk number "+str(chunk_number))
    plt.plot([i for i, x in enumerate(f1) if i % step == 0],[x for i, x in enumerate(f1) if i % step == 0], color="red",label="F1_Score")
    plt.plot([i for i, x in enumerate(ac) if i % step == 0],[x for i, x in enumerate(ac) if i % step == 0], color="blue",label="Accurce_score")
    plt.plot([i for i, x in enumerate(pr) if i % step == 0],[x for i, x in enumerate(pr) if i % step == 0], color="yellow",label="Precision_score")
    plt.plot([i for i, x in enumerate(rc) if i % step == 0],[x for i, x in enumerate(rc) if i % step == 0], color="green",label="Recall_score")

    plt.legend(loc="best")
    plt.xlabel('iteration')
    plt.ylabel('results per iteration')
    plt.xscale('log',base=2)

    # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Tabu_Search_results_for_chunk_number_"+str(chunk_number)))
    # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Tabu_Search_results_for_chunk_number_"+str(chunk_number)+'.svg'),format='svg')
    plt.show()

In [ ]:
import matplotlib.ticker as mticker

def plot_results(metric_name, methods_name, colors, step=1):
    results = finder(metric_name)
    plt.figure(figsize=(15, 5))
    plt.title(metric_name)
    for k, result in enumerate(results):
        x = [i for i, x in enumerate(results[k]) if i % step == 0]
        y = [x for i, x in enumerate(results[k]) if i % step == 0]
        plt.plot(x, y, color=colors[k], label=methods_name[k])
        plt.scatter(x, y, color=colors[k], s=20)  # Add dots to the plot

    plt.legend(loc="best")
    plt.xlabel('Chunk number')
    plt.ylabel('Results per Chunk')

    # Format x-axis ticks as integers
    plt.xticks(rotation=90)
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(step))
    plt.gca().xaxis.set_major_formatter(plt.ScalarFormatter(useOffset=False, useMathText=True))

    # plt.savefig(os.path.join(os.path.join(results_path, 'Imges'), metric_name))
    # plt.savefig(os.path.join(os.path.join(results_path, 'Imges'), metric_name+'.svg'), format='svg')
    plt.show()

In [ ]:
def plot_dimensional_reduction(tabu,softmax,single_agent,average):
  # set width of bar
  barWidth = 0.05
  fig = plt.subplots(figsize =(8, 5))
  # Set position of bar on X axis
  br1 = np.arange(len(tabu))
  br2 = [x + barWidth for x in br1]
  br3 = [x + barWidth for x in br2]
  br4 = [x + barWidth for x in br3]
  # Make the plot

  plt.bar(br1, tabu, color ='r', width = barWidth,edgecolor ='grey', label ='2TS-DFS')
  plt.bar(br2, softmax, color ='g', width = barWidth,edgecolor ='grey', label ='ALC')
  plt.bar(br3, single_agent, color ='b', width = barWidth,edgecolor ='grey', label ='OA-OT')
  plt.bar(br4, average, color ='y', width = barWidth,edgecolor ='grey', label ='AC')
  # Adding Xticks
  plt.xlabel('Chunk number', fontweight ='bold', fontsize = 10)
  plt.ylabel('The remaining dimensions after being reduced from 100', fontweight ='bold', fontsize = 8)
  plt.xticks([r + barWidth for r in range(len(number_of_chunk))],[str(i) for i,_ in enumerate(number_of_chunk)])
  plt.legend(loc="best")
  # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Dimensional reduction amount"))
  # plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Dimensional reduction amount.svg"),format='svg')
  plt.show()

In [ ]:
def PointsInCircum(r,n=100):
    return [(math.cos(2*math.pi/n*x)*r+np.random.normal(-30,30),math.sin(2*math.pi/n*x)*r+np.random.normal(-30,30)) for x in range(1,n+1)]

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
import pandas as pd

def plot_conf_matrix(true, pred, classes):
    cf = confusion_matrix(true, pred)
    plt.figure(figsize=(12,10))
    sns.set(font_scale=1.4)
    sns.heatmap(np.round(cf / cf.sum(axis=0), 3).T, annot=True, annot_kws={"size": 16},fmt='g',
                cmap='Blues', xticklabels = classes ,yticklabels = classes)
    # plt.savefig('confusion_matrix.png')
    # plt.savefig('confusion_matrix.svg')

    plt.show()
# plot_conf_matrix(labels, y_preds, class_names)

In [ ]:
def get_callbacks():
  filepath = os.path.join(model_info_path,'Tabu_Search_'+str(num)+'_.best.hdf5')
  history_logger = CSVLogger(os.path.join(model_info_path,'history'+str(num)+'.csv'), separator=",", append=True)
  checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
  es = EarlyStopping(monitor='val_loss', patience=5)
  callbacks_list = [GarbageCollectorCallback(),es,history_logger,checkpoint] # ,,
  return callbacks_list

In [ ]:
def evaluate_result(y_predict,y_true):
    y_predict[y_predict>0.5] = 1
    y_true = list(map(_replaceitem, y_true))
    y_predict = list(map(_replaceitem, y_predict))
    return f1_score(y_true=y_true,y_pred=y_predict),accuracy_score(y_true=y_true,y_pred=y_predict),precision_score(y_true=y_true,y_pred=y_predict),recall_score(y_true=y_true,y_pred=y_predict)

In [ ]:
def chunking_data(NUMBER):
    """
    _INPUT (Data) CHUNK OF DATASET (test_size=0.20) SIZE OF SPLITNG
    _OUTPUT (X_train , y_train , X_test , y_test)
    """
    x_chunk , y_chunk = load_object('X_train_hiden_for_chunk_'+str(NUMBER),par_path) , load_object('y_train_hiden_for_chunk_'+str(NUMBER),par_path)
    X_new_chunks , y_new_chunks = np.array(np.split(x_chunk, x_chunk.shape[0]/100)) , np.array(np.split(y_chunk, y_chunk.shape[0]/100))
    return X_new_chunks[:-1] , y_new_chunks[:-1] , X_new_chunks[-1] , y_new_chunks[-1]

In [ ]:
def finder(metric):
  result = pd.read_csv(os.path.join(os.path.join(results_path,'Imges'),"Numerical results.csv"))
  return  [result[result['Method_name']=='2TS-DFS'][metric].tolist(),
          result[result['Method_name']=='ALC'][metric].tolist(),
          result[result['Method_name']=='OA-OT'][metric].tolist(),
          result[result['Method_name']=='AC'][metric].tolist(),
          result[result['Method_name']=='JSContana-S'][metric].tolist(),
          result[result['Method_name']=='JSContana'][metric].tolist()]

In [ ]:
def apply_change_on_data(X,mask,change):
  if change[0] is None:
    return apply_mask(X,mask),mask
  mask[change[0]][change[1]] = 1-mask[change[0]][change[1]]
  return apply_mask(X.copy(),mask.copy()),mask.copy()

In [ ]:
def check_aspiration_criteria(f1,ac):
  f1_aspiration_criteria,ac_aspiration_criteria= 0.80,0.80
  if(f1>=f1_aspiration_criteria)and(ac>=ac_aspiration_criteria):return True
  else:return False

In [ ]:
def get_similarity(X_test,X_train):
  similarites=[]
  for x in X_test:
    similarity=[]
    for X in X_train:
      similarity.append(np.mean(np.sum(x*X, axis=1)/(norm(x, axis=1)*norm(X, axis=1))))
    similarites.append(similarity.index(max(similarity)))
  return similarites

In [ ]:
def get_euclidean_distance(X_test,X_train,cluster_train):
  distances = []
  for x_test in X_test:
    distanc = []
    for x_train in X_train:
      distanc.append(np.sum(np.square(x_test - x_train)))
    distances.append(distanc.index(min(distanc)))
  return cluster_train[distances]

In [ ]:
def update_tabu_list(solution):
  Tabu_list.append(solution)
  value_of_update = len(Tabu_list)-Tabu_tenur
  if value_of_update>0:
    del Tabu_list[:value_of_update]
  return Tabu_list

In [ ]:
def apply_mask(X,mask,cluster=None):
  if cluster is None:
    for i,x in enumerate(X):
      X[i] = x*mask
  else:
    for i,x in enumerate(X):
      X[i] = x*mask[cluster[i]]
  return X

In [ ]:
def get_statistical_information(X):
  new_x = []
  for x in X:
    new_x.append(np.array([np.ndarray.min(x),np.ndarray.max(x),np.ndarray.mean(x),np.ndarray.std(x),np.median(x),np.var(x)]))
  return np.array(new_x)

In [ ]:
def save(i):
  save_object(solutions, 'solutions_'+str(i),results_path)
  save_object(Precision_test, 'Precision_test_'+str(i),results_path)
  save_object(Recall_test, 'Recall_test_'+str(i),results_path)
  save_object(F1_test, 'F1_test_'+str(i),results_path)
  save_object(Ac_test, 'Ac_test_'+str(i),results_path)
  save_object(Tabu_list, 'Tabu_list_'+str(i),results_path)
  save_object(cluster_train, 'cluster_train_'+str(i),results_path)
  save_object(cluster_test, 'cluster_test_'+str(i),results_path)
  save_object(F1_train, 'F1_train_'+str(i),results_path)
  save_object(Ac_train, 'Ac_train_'+str(i),results_path)
  save_object(Precision_test, 'Precision_train_'+str(i),results_path)
  save_object(Recall_test, 'Recall_train_'+str(i),results_path)
  save_object(agglomerativeClustering, 'agglomerativeClustering_'+str(i),results_path)
  save_object(y_test_predict, 'y_test_predict'+str(i),results_path)
  save_object(y_train_predict, 'y_train_predict'+str(i),results_path)

In [ ]:
def load(i):
  F1_train = load_object('Precision_test_'+str(i),results_path)
  F1_test = load_object('F1_test_'+str(i),results_path)
  Ac_train = load_object('Recall_test_'+str(i),results_path)
  Ac_test = load_object('Ac_test_'+str(i),results_path)
  return F1_test,Ac_test,Recall_test,Precision_test

In [ ]:
def neighbour_solution(solution,e=False):
  if e:
    new_solution = np.random.randint(low=0,high=2,size=(number_of_calss,X_train.shape[2]))
  else:
    new_solution = solution.copy()
    for i,_ in enumerate(new_solution):
      neighbours = np.random.randint(low=0,high=100,size=5)
      for j in neighbours:
        new_solution[i][j] = 1-solution[i][j]
  return new_solution

In [ ]:
def Tabu_list_checker(solution):
  for t in Tabu_list:
    if np.all(np.equal(solution,t)):
      return True
  return False

In [ ]:
def generate_neighbour_solution(solution,e=False):
  neighbour = neighbour_solution(solution,e)
  while True:
    t = Tabu_list_checker(neighbour)
    if t:neighbour = neighbour_solution(solution,e)
    else:break
  return neighbour

## Tabu Search Algorithm

In [ ]:
number_of_calss = 35
Tabu_tenur,Tabu_list,num=100,[],0
number_of_chunk =[i[:-4] for i in os.listdir(par_path)[::-1] if i[:7]=='X_train']
number_of_chunk = number_of_chunk[:-1]
agglomerativeClustering = AgglomerativeClustering(n_clusters=number_of_calss, affinity='euclidean')

In [ ]:
for i,x in tqdm(enumerate(number_of_chunk)):
  X_train , y_train = load_object('X_train_hiden_for_chunk_'+str(i),par_path) , load_object('y_train_hiden_for_chunk_'+str(i),par_path)
  X_test , y_test = load_object('X_test_hiden_for_chunk_'+str(i),par_path) , load_object('y_test_hiden_for_chunk_'+str(i),par_path)
  number_of_iteration,num = X_train.shape[2]*10,i
  if i==0:
    model = create_model(shape=X_train[0].shape,without_first_layer=True,i=i)
    model.save(os.path.join(model_info_path,"tabu_search_model.h5"))
  else:
    Tabu_list = load_object('Tabu_list_'+str(i-1),results_path)
    agglomerativeClustering = load_object('agglomerativeClustering',results_path)
    model = load_model(os.path.join(model_info_path,"tabu_search_model.h5"))
  cluster_train = agglomerativeClustering.fit_predict(get_statistical_information(X_train))
  cluster_test = get_euclidean_distance(X_test,X_train,cluster_train)
  current_solution = np.random.randint(low=0,high=2,size=(number_of_calss,X_train.shape[2]))
  solutions,F1_train,Ac_train,F1_test,Ac_test,Precision_test,Recall_test,Precision_train,Recall_train = [],[],[],[],[],[],[],[],[]
  for step in tqdm(range(number_of_iteration)):
    x_train = X_train.copy()
    x_train = apply_mask(x_train,current_solution,cluster_train)
    history = model.fit(x_train, y_train,epochs=10,batch_size=100,validation_split=0.20,callbacks=get_callbacks(), verbose=0)
    x_test = X_test.copy()
    x_test = apply_mask(x_test,current_solution,cluster_test)
    y_test_predict = model.predict(x_test, verbose=0)
    f1_test,ac_test,precision_test,recall_test =  evaluate_result(y_test_predict,y_test)
    y_train_predict = model.predict(x_train, verbose=0)
    f1_train,ac_train,precision_train,recall_tain =  evaluate_result(y_train_predict,y_train)

    Tabu_list = update_tabu_list(current_solution)
    aspiration_criteria = check_aspiration_criteria(f1_test,ac_test)
    if not aspiration_criteria:
      del model
      model = load_model(os.path.join(model_info_path,"tabu_search_model.h5"))
    else:
      model.save(os.path.join(model_info_path,"tabu_search_model.h5"))

    solutions.append(current_solution)
    Precision_test.append(precision_test)
    F1_test.append(f1_test)
    Recall_test.append(recall_test)
    Ac_test.append(ac_test)

    Precision_train.append(precision_train)
    F1_train.append(f1_train)
    Recall_train.append(recall_tain)
    Ac_train.append(ac_train)

    save(i)
    e = True if max(F1_test)<97 else False
    current_solution = generate_neighbour_solution(current_solution.copy(),e)
    if f1_test>=97:break

In [ ]:
for i in range(len(number_of_chunk)):
  solution = load_object('best_solution__'+str(i),results_path)
  sums = []
  for s in solution:
    sums.append(sum(s))
  selected_features_for_each_cluster(sums,i)

In [ ]:
_tabu = load_object('_tabu',results_path)
_softmax = load_object('_softmax',results_path)
_single_agent = load_object('_single_agent',results_path)
_average = load_object('_average',results_path)
plot_dimensional_reduction(_tabu,_softmax,_single_agent,_average)

In [ ]:
methods_name = ['2TS-DFS','ALC','OA-OT','AC','JSContana-S','JSContana']
colors = ['r', 'g', 'b', 'm', 'c','y']

In [ ]:
plot_results('Accuracy_score',methods_name,colors,step=1)

In [ ]:
plot_results('Precision_score',methods_name,colors,step=1)

In [ ]:
plot_results('Recall_score',methods_name,colors,step=1)

In [ ]:
plot_results('F1_score',methods_name,colors,step=1)

In [ ]:
plot_results('Prediction_Time',methods_name,colors,step=1)

In [ ]:
plot_results('Binary Cross-Entropy',methods_name,colors,step=1)

In [ ]:
plot_results('Mean Absolute Error',methods_name,colors,step=1)

In [ ]:
plot_results('Mean Squared Error',methods_name,colors,step=1)

In [ ]:
plot_results('False Positive Rate',methods_name,colors,step=1)

In [ ]:
def loader(i):
  F1_train = load_object('_F1_train'+str(i),results_path)
  F1_test = load_object('_F1_test'+str(i),results_path)
  Ac_train = load_object('_Ac_train'+str(i),results_path)
  Ac_test = load_object('_Ac_test'+str(i),results_path)
  return F1_train,F1_test,Ac_train,Ac_test

In [ ]:
def find_factors(number):
    factors = []
    for i in range(1, number + 1):
        if number % i == 0:
            factors.append(i)
    return factors

In [ ]:
for i in range(4):
  F1_train,F1_test,Ac_train,Ac_test = loader(i)
  _plot_results_for_chunk_number(i,f1=F1_test,ac=Ac_test,pr=F1_train,rc=Ac_train)